### Exercise 1: Autocomplete using the trie datastructure

A trie (also called prefix tree) is a versatile data structure, which often comes in handy in text processing. It encodes a set of strings in a very compact format. The tree structure makes certain operations that are prohibitively expensive with a standard hash table, such as the one used in the Python `dict`, fast and easy. For instance, listing all strings that begin with a given string, or finding the *best* (e.g. most common) string with a given prefix. 

In this exercise you will implement an auto-complete facility that suggests the completion $c$ with the highest score conditioned on the query $q$. In other words, you should find the completion that maximizes $\text{score}(c|q)$, as written below:

$$
\arg \max_{c} \text{score}(c|q)
$$

This will simply be the string with the highest associated score in the subtree rooted at $q$.


#### Dataset

You will be working with a small toy data set as well as a larger AOL data set derived from real search queries. The AOL data set is surrounded by some controversy, because shortly after its release in 2006 researchers found that the user anonymization was insufficient, and people's private information leaked through the query strings, including credit card numbers. To deal with the privacy issues we only include high-frequency queries isssued by many people in our version of the data set.

Download the data from [here](https://dl.dropboxusercontent.com/u/1423772/sciprog/aol-01-top5k.txt) and place it in the current folder.

In [1]:
%matplotlib inline
import networkx as nx

class TrieNode(object):
    
    def __init__(self, value, count=0):
        self.value = value
        self.first_child = None
        self.next_sibling = None
        self.max_count = count
        
    def add(self, name, count=0):
        """Adds `name` to the subtree rooted at this node.
        
        When you have completed Ex. 1.2, this function will check and possibly update the `max_count` attribute
        of the current node as well as nodes on the path to `name` in the subtree.
        """
        found = None
        for child in self.children():
            if child.value == prefix:
                found = child
                break
        
        if not found:
            found = TrieNode(prefix, count)
            if self.first_child:
                found.next_sibling = self.first_child
            self.first_child = found
            
        if len(name) > 1:
            found.add(name[1:], count)
            
    def find(self, name, prefix=''):
        """
        Returns the longest match for `name`, as well as the matching node.
        """
        if name[0] != self.value:
            return prefix, None

        current_name = prefix + self.value
        
        if len(name) > 1:
            for child in self.children():
                if child.value == name[1]:
                    return child.find(name[1:], prefix + self.value)
            
        return current_name, self
        
    def children(self):
        """
        Enumerates the children of the node.
        """
        child = self.first_child
        while child:
            yield child
            child = child.next_sibling

        
    # Visualization functions
    #
    def draw(self):
        G = self.to_networkx()
        pos = nx.graphviz_layout(G, prog='dot')
        nx.draw(G, pos, with_labels=True, arrows=False, edge_color='gray', alpha=1, font_size=15, node_color='white',
                labels=dict((n, n[-1]) for n in G.nodes()))
    
    def to_networkx(self):
        G = nx.DiGraph()
        to_visit = []
        
        # DFS traversal
        to_visit += [(self.value, child) for child in self.children()]
        
        while to_visit:
            parent_prefix, next_node = to_visit.pop()
            prefix = parent_prefix + next_node.value
            G.add_edge(parent_prefix, prefix)

            to_visit += [(prefix, child) for child in next_node.children()]

        return G

    def __repr__(self):
        return "{}:{}".format(self.value, self.max_count)

### Ex 1.1 Displaying the trie

Add the following strings to the trie and display it using `draw()`.

```
aardvark
talent
tale
talcum
taleban
```

In [ ]:
# Your code here

### Ex 1.2 String scoring

Make a new trie containing the same set of strings as in the previous exercise, only now each string is associated with an integer value indicating the count or scoring of that string in a corpus. 

The idea is to let the `max_count` attribute of a given node reflect the maximum score of any string in its subtree, because that enables quick retrieval of the best scoring string.

You will have to modify the `add` method to do this. 

Here is the table of strings and values:

```
aardvark     10
talent        9
tale          7
talcum        2
taleban       8
```

As a sanity check of your implementation, you should verify that `max_count` is 10 for the root node *^*, 9 for *^tal*, and 2 for *^talc*.

In [ ]:
# Your code here

### Ex 1.3: Autocompletion

You are now ready to write the `autocomplete` function. The function should be flexible enough that you can use it with different tries. During development it is often most convenient to work with a small "toy" data set that you can print to the screen and where you can accuractely predict what the results are going to look like. Once you have arrived at an implementation you trust, it is easy to switch to the larger data set. 

We suggest that you create an `autocomplete` function with the following three parameters:

1. The trie represented by the root node;
2. The query as a string; and
3. An `n_best` integer parameter specifying the number of suggestions to give back. 

Finding the best completion is similar to graph search, although actually a little less complicated, since you never risk visiting the same node more than once in a tree. Additionally, the precomputed `max_count` attribute is extremely helpful in guiding the search. To find the single best completion, you merely have to always follow the path with the highest `max_count`. Note, though, that you must keep track of the path to discover what the completion is, since that information is not stored in the goal node. 

#### (Advanced) Retrieving n-best

Returning the $n$ best completions, instead of the single best, can be accomplished using a variation of the A\* algorithm. Nodes are added to a priority queue with a priority of `-max_count` and visited in order. A completion is finished when a node with the final symbol *\$* is encountered. Thus the algorithm should terminate when `n_best` nodes with a `value` of *\$* have been explored.

In [3]:
# Efficient priority queue implementation 
# from Python standard library
from queue import PriorityQueue

### Ex. 1.4: A large trie

Read in the `aol-01-top5k.txt` file, adding the queries and counts to the trie. The file is created as the output of the unix utility `uniq`.

Try different queries and see what comes up. Do you think this is the sort of data that Google uses for their auto completion? How could the autocompleter be improved --- algorithmically and with respect to data?

And welcome to 2006. It's good to have you back!

In [ ]:
# Your code here

### Ex. 1.5 (optional AND advanced): Create a "Did you mean"-spelling corrector

The spelling corrector should be able to deal with single substitution errors (one letter swapped for another) and insertion errors (putting extra letters in).

In [ ]:
# Your code here